In [1]:
import pandas as pd
import numpy as np

import sys

from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo

import matplotlib.pyplot as plt
import geopandas as gpd
import xlrd
import plotly.express as px
import plotly.graph_objects as go

import json

In [2]:
# Load the data to be used
#%run "...py"

# ScaleWAYS - Agro-economic Model - v1

## 1. Overview 

The scaleWAYS project aims to identify the the most suitable regions in the extended Lake Victoria Basin (eLVB) for producing rice . Increasing production of rice in the eLVB is seen as a good business opportunity given that demand for this staple is growing in the region and that a great proportion of this cereal comes from Asia. Increasing domestic production can contribute to food security, specially in situations like pandemics where food value chain are disrupted. 

The rapid growth in rice demand calls for improving productivity and resources use efficiency in agriculture. Production of rice are specially known for their negative impacts on the environment (e.g., greenhouse gases emissions, intensive water use), which also calls for improving productivity sustainably. Therefore, simulating the yields and environmental impacts of different agronomic options under different climate change and economic scenarios can provide rich insights of which options are the most sustainable to intensify the production of this crop.

A systems and modeling approach can be helpful to identify the best strategies to meet the growing demand for rice, specially under the uncertainty of climate change. 

### 1.1 Agro-economic model 
The objective of the agro-economic model is to find the most profitable 
combination of production options that will satisfy the rice demand of the extended Lake Victoria Basin. Four combinations of 2 levels of intensification (i.e., low and intermerdiate use of inputs) and rainfed and irrigated production are explored. The upper limits are defined by the land, water, and labor availability; and the lower limits are defined by rice demand.


---


**Sets** 
\begin{eqnarray*}
    I  =  \{ \text{low inputs, intermediate inputs} \} \\
    W  =  \{ \text{rainfed, irrigated} \} 
\end{eqnarray*}

**Parameters**
\begin{eqnarray*}
    & p  & \textrm{price of crop}  \\
    & y_{i,w} & \textrm{yields}  \\
    & pc_{i,w} & \textrm{production cost per unit of} \\
    & w & \textrm{irrigation water requirements per unit of area} \\
    & \eta & \textrm{irrigation efficiency} \\
    & l_{i,w} & \textrm{labor requirement per unit of area} \\
    & land_{i,w} & \textrm{land suitable and available} \\
    & water & \textrm{water available} \\
    & labor & \textrm{labor available} \\
    & demand & \textrm{demand for rice}
\end{eqnarray*}

**Variables**
\begin{eqnarray*} 
    & area_{i,w} & \text{area allocated for rice production} \\
    & irr_{i,w} & \text{irrigation water requirement} \\
    & prod_{i,w} & \text{crop production} \\
    & profit_{i,w} & \text{profit}
 \end{eqnarray*}
 
**Maximize**
 \begin{align}
& \displaystyle \sum_{i,w} (y_{i,w} * area_{i,w} * p) - pc_{i,w} & \textrm{maximize profit}
\end{align}

**Constraints**
\begin{align}
& \sum_{i,w} area_{i,w} \leq \sum_{i,w} land_{i,w} \\ 
& \sum_{i,w} irr_{i,w} \leq \eta \cdot water  \label{eq:wateravailable} \\ 
& \sum_{i,w} l_{i,w} \cdot area_{i,w} \leq labor  \label{eq:laboravailable} \\ 
& \sum_{i,w} prod_{i,w} \leq demand  \label{eq:ricedemand} \\ 
& area_{i,w} \geq 0 \label{eq:nonnegativearea}
\end{align}

---

### 1.2 Data inputs

#### 1.2.1 Area and yields data

$land_{i,w}$ suitable for producing rice, and expected yields ($y_{i,w}$) can be quantified and located using the outputs of the GAEZ model. Yet, these data has to be processed to fulfill some of the assumptions defined in teh scenarios. For example, in the "conservation" scenario, we may limit rice and fodder production to actual cropland and grassland. This hypothetically will prevent future deforestation and degradation of other ecosystems, such as wetlands. 

GAEZ estimations for current harvested area (cropland) can be validated with (i) remotely sensed data, and (ii) data from other models, such as SPAM and CELL5C from IFPRI. This is speciall important 

#### 1.2.2 Water data

$water$ available for production of rice, fodder and crops can be estimated from the outputs of ECHO and cWATm. In the previous study, our colleagues calculated water withdrawals from different sectors (i.e., agriculture, domestic, industry, livestock) and environmental flows (2006-2055, for the 61 watersheds) using two different climate models: HadGEM2 and MIROC5. 

For the EGU presentation, I calculated the water available for irrigation as follows:

\begin{eqnarray}
  & ww = \frac{1}{j} \sum_{ij} ww_{ij} & \textrm{withdrawal from sector $i$ under climate scenario $j$} \label{eq:waterwithdrawal} \\
  & env = \frac{1}{j} \sum_{j} env_{j} & \textrm{environmental flow estimated $i$ under climate scenario $j$} \\
  & dis = \frac{1}{j} \sum_{j} dis_{j} & \textrm{discharge estimated under climate scenario $j$} \\
  & wat = dis - ( ww + env) & \textrm{water available for irrigation}
\end{eqnarray}{}

#### 1.2.3 Economic data (prices, production costs)

#### 1.2.4 Demographic and food consumption data